# Make training dataset

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
pd.set_option('display.max_columns', 500)

In [ ]:
train = pd.read_csv('../first_50000_rows.csv', sep='|')
train = train.drop(['portId','etaRaw'], axis=1)
train['time'] = pd.to_datetime(train['time'])
train = train.sort_values(by=['vesselId','time'])
train.head()

In [ ]:
print(train["vesselId"].value_counts())

# modify train so it only contains vesselId: 6323f2287abc89c0a9631e57 and 61e9f466b937134a3c4c0273

#train = train[train['vesselId'].isin(['6323f2287abc89c0a9631e57', '61e9f466b937134a3c4c0273'])]

In [ ]:
vessels = pd.read_csv('../vessels.csv', sep='|')
display(vessels.head())

# Keep only the vessels that are in the train set
print(vessels.shape)
vessels = vessels[vessels['vesselId'].isin(train['vesselId'])]
print(vessels.shape)
print(vessels.describe())

# print number of nan values in each column
print(vessels.isna().sum())

In [ ]:
print(vessels['shippingLineId'].value_counts())

In [8]:
# Merge vessels into train
train = pd.merge(train, vessels, on='vesselId', how='left')
train.drop(['NT','depth', 'draft', 'freshWater', 'fuel', 'homePort', 'maxHeight','maxSpeed', 'maxWidth', 'rampCapacity','shippingLineId'], axis=1, inplace=True)
train.fillna(-1, inplace=True)


In [ ]:
train.head()

In [10]:
def create_training_data(df: pd.DataFrame, N_KEEP_PAST: int) -> pd.DataFrame:
    data_rows = []  # List to collect all the data rows
    vessel_list = df['vesselId'].unique()
    for vessel in tqdm(vessel_list):
        vessel_data = df[df['vesselId'] == vessel].sort_values(by='time').reset_index(drop=True)
        num_rows = vessel_data.shape[0]
        if num_rows <= N_KEEP_PAST:
            continue  # Skip vessels with insufficient data
        for i in range(N_KEEP_PAST, num_rows):
            # Collect past N_KEEP_PAST locations and timestamps
            past_data = vessel_data.loc[i - N_KEEP_PAST:i - 1].reset_index(drop=True)
            current_data = vessel_data.loc[i]
            # Prepare a dictionary to hold the features and target
            data_row = {}
            target_time = current_data['time']
            for j in range(N_KEEP_PAST):
                past_time = past_data.loc[j, 'time']
                # Calculate the difference in minutes between past time and target time
                time_diff = (target_time - past_time).total_seconds() / 60.0  # Difference in minutes
                data_row[f'minutes_from_target_{j}'] = time_diff
                data_row[f'lat_{j}'] = past_data.loc[j, 'latitude']
                data_row[f'lon_{j}'] = past_data.loc[j, 'longitude']
                #######
                data_row[f'cog_{j}'] = past_data.loc[j, 'cog']
                data_row[f'sog_{j}'] = past_data.loc[j, 'sog']
                data_row[f'rot_{j}'] = past_data.loc[j, 'rot']
                data_row[f'heading_{j}'] = past_data.loc[j, 'heading']
                data_row[f'navstat_{j}'] = past_data.loc[j, 'navstat']
                ######
            # Add current location as the target
            data_row['target_lat'] = current_data['latitude']
            data_row['target_lon'] = current_data['longitude']
            #data_row['CEU'] = current_data['CEU']
            #data_row['DWT'] = current_data['DWT']
            #data_row['GT'] = current_data['GT']
            #data_row['vesselType'] = current_data['vesselType']
            #data_row['breadth'] = current_data['breadth']
            #data_row['enginePower'] = current_data['enginePower']
            #data_row['length'] = current_data['length']
            #data_row['yearBuilt'] = current_data['yearBuilt']
            
            data_row['vesselId'] = vessel  # Include vesselId if needed
            data_row['target_time'] = target_time # Remove eventually
            
            
            # Append the row to the list
            data_rows.append(data_row)
    # Create final DataFrame from the list of data rows
    final_df = pd.DataFrame(data_rows)
    return final_df

In [ ]:
N_KEEP_PAST = 5
final_train = create_training_data(train, N_KEEP_PAST)

In [ ]:
print(final_train.shape)
display(final_train.tail())

# Modify Test dataset

In [14]:
test = pd.read_csv('../ais_test.csv')
test['time'] = pd.to_datetime(test['time'])

In [ ]:
print(test.shape)
# modify test so it only contains vesselId: 6323f2287abc89c0a9631e57 and 61e9f466b937134a3c4c0273
#test = test[test['vesselId'].isin(['6323f2287abc89c0a9631e57', '61e9f466b937134a3c4c0273'])]
print(test.shape)
display(test.head())

In [16]:
def create_test_features(train_df: pd.DataFrame, test_df: pd.DataFrame, N_KEEP_PAST: int) -> pd.DataFrame:
    data_rows = []  # List to collect all the data rows
    vessel_list = test_df['vesselId'].unique()
    
    for vessel in tqdm(vessel_list):
        # Get the test data for this vessel
        test_vessel_data = test_df[test_df['vesselId'] == vessel]
        for idx, test_row in test_vessel_data.iterrows():
            target_time = test_row['time']
            ID = test_row['ID']
            scaling_factor = test_row['scaling_factor']  # If needed later
            
            # Get past data from train for this vessel before the target time
            vessel_train_data = train_df[(train_df['vesselId'] == vessel) & (train_df['time'] < target_time)]
            vessel_train_data = vessel_train_data.sort_values(by='time').reset_index(drop=True)
            num_past_points = vessel_train_data.shape[0]
            
            if num_past_points < N_KEEP_PAST:
                # Not enough past data; decide how to handle (skip or pad with NaNs)
                continue  # Or handle as per your requirement
            
            # Get the last N_KEEP_PAST records
            past_data = vessel_train_data.iloc[-N_KEEP_PAST:].reset_index(drop=True)
            
            # Prepare a dictionary to hold the features
            data_row = {}
            for j in range(N_KEEP_PAST):
                past_time = past_data.loc[j, 'time']
                # Calculate the difference in minutes between past time and target time
                time_diff = (target_time - past_time).total_seconds() / 60.0  # Difference in minutes
                data_row[f'minutes_from_target_{j}'] = time_diff
                data_row[f'lat_{j}'] = past_data.loc[j, 'latitude']
                data_row[f'lon_{j}'] = past_data.loc[j, 'longitude']
                ###
                data_row[f'cog_{j}'] = past_data.loc[j, 'cog']
                data_row[f'sog_{j}'] = past_data.loc[j, 'sog']
                data_row[f'rot_{j}'] = past_data.loc[j, 'rot']
                data_row[f'heading_{j}'] = past_data.loc[j, 'heading']
                data_row[f'navstat_{j}'] = past_data.loc[j, 'navstat']
                ###
                
                #if j == N_KEEP_PAST - 1:
                #    data_row['CEU'] = past_data.loc[j, 'CEU']
                #    data_row['DWT'] = past_data.loc[j, 'DWT']
                #    data_row['GT'] = past_data.loc[j, 'GT']
                #    data_row['vesselType'] = past_data.loc[j, 'vesselType']
                #    data_row['breadth'] = past_data.loc[j, 'breadth']
                #    data_row['enginePower'] = past_data.loc[j, 'enginePower']
                #    data_row['length'] = past_data.loc[j, 'length']
                #    data_row['yearBuilt'] = past_data.loc[j, 'yearBuilt']
            
            
            # Include vesselId and ID for result matching
            data_row['vesselId'] = vessel
            data_row['ID'] = ID
            # Append the row to the list
            data_rows.append(data_row)
    
    # Create test features DataFrame from the list of data rows
    test_features = pd.DataFrame(data_rows)
    return test_features

In [ ]:
test_final = create_test_features(train, test, N_KEEP_PAST)

In [ ]:
display(test_final.head())

In [19]:
final_train.to_csv('final_train.csv', index=False)
test_final.to_csv('test_final.csv', index=False)

# Machine learning part

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import Ridge
from scipy.optimize import minimize
import catboost as cb
from sklearn.ensemble import HistGradientBoostingRegressor

# Load the data
test_final = pd.read_csv('test_final.csv')
final_train = pd.read_csv('final_train.csv')


print("Initial shape of training data:", final_train.shape)
final_train = final_train.dropna()
print("Shape after dropping NaNs:", final_train.shape)


# Define features and targets
features = final_train.drop(columns=['target_lat', 'target_lon', 'vesselId', 'target_time'])
targets = final_train[['target_lat', 'target_lon']]

feature_columns = features.columns
X_test = test_final[feature_columns]

# Define the models
models = {
    'ridge_optuna': MultiOutputRegressor(Ridge(alpha=1432.7819410601255, random_state=42)),
    'lightgbm_optuna': MultiOutputRegressor(lgb.LGBMRegressor(n_estimators=82, learning_rate=0.1000124709018958, num_leaves=256, max_depth=12, min_child_samples=6, subsample=0.7895654034356272, colsample_bytree=0.705074943165495, reg_alpha=0.965623735511052, reg_lambda=0.9415109979963966, verbose=-1, random_state=42, n_jobs=-1)),
    'catboost_optuna': MultiOutputRegressor(cb.CatBoostRegressor(iterations=476, learning_rate=0.1596674384872822, depth=10, l2_leaf_reg=3.646475024083615, bagging_temperature=0.12986428988678064, random_state=42, verbose=0, thread_count=-1)),
    'histgradientboosting_optuna': MultiOutputRegressor(HistGradientBoostingRegressor(learning_rate=0.11343690296237537, max_iter=456, max_leaf_nodes=157, max_depth=13, min_samples_leaf=31, l2_regularization=0.24506344544473505, max_bins=240, random_state=42, verbose=0)),
    'xgboost_optuna': MultiOutputRegressor(xgb.XGBRegressor(n_estimators=422, max_depth=14, learning_rate=0.02124556170140244, subsample=0.614092688023698, colsample_bytree=0.5846203682106957, gamma=0.7660348189459568, reg_alpha=0.5096703756053615, reg_lambda=0.5556686221134866, objective='reg:squarederror', random_state=42, n_jobs=-1)),
}

# Prepare arrays to hold OOF predictions and test predictions
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_preds = {model_name: np.zeros((features.shape[0], targets.shape[1])) for model_name in models}
test_preds_cv = {model_name: np.zeros((X_test.shape[0], targets.shape[1], n_splits)) for model_name in models}

# Perform cross-validation and collect predictions
for fold, (train_idx, valid_idx) in enumerate(kf.split(features, targets)):
    print(f"\nFold {fold + 1}/{n_splits}")
    X_train, y_train = features.iloc[train_idx], targets.iloc[train_idx]
    X_valid, y_valid = features.iloc[valid_idx], targets.iloc[valid_idx]
    
    for model_name, model in models.items():
        print(f"Training and predicting with model: {model_name}")
        clf = model
        clf.fit(X_train, y_train)
        y_pred_valid = clf.predict(X_valid)
        y_pred_test = clf.predict(X_test)
        
        # Save OOF predictions
        oof_preds[model_name][valid_idx] = y_pred_valid
        # Save test predictions
        test_preds_cv[model_name][:, :, fold] = y_pred_test

# Define the loss function for optimization
def mse_loss(weights):
    weights = np.array(weights)
    # Normalize weights to sum to 1
    weights = weights / np.sum(weights)
    # Combine OOF predictions using the weights
    final_oof = np.zeros_like(targets.values)
    for i, model_name in enumerate(models):
        final_oof += weights[i] * oof_preds[model_name]
    # Compute mean squared error
    mse = mean_squared_error(targets.values, final_oof)
    return mse

# Optimization methods to try
methods = [
    'Nelder-Mead', 'Powell', 'CG', 'BFGS',
    'L-BFGS-B', 'TNC', 'SLSQP',
]

# Initial weights
initial_weights = np.ones(len(models)) / len(models)

# Constraints and bounds
constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
bounds = [(0, 1)] * len(models)

# Optimize weights using different methods
best_mse = np.inf
best_weights = None
best_method = None

for method in methods:
    print(f"\nOptimizing weights using method: {method}")
    try:
        if method in ['SLSQP']:
            res = minimize(mse_loss, initial_weights, method=method, bounds=bounds, constraints=constraints)
        elif method in ['L-BFGS-B', 'TNC']:
            res = minimize(mse_loss, initial_weights, method=method, bounds=bounds)
        else:
            # For unconstrained methods, weights will be normalized in mse_loss
            res = minimize(mse_loss, initial_weights, method=method)
        if res.fun < best_mse:
            best_mse = res.fun
            best_weights = res.x / np.sum(res.x)  # Normalize weights
            best_method = method
        print(f"Method: {method}, MSE: {res.fun}")
    except Exception as e:
        print(f"Method: {method}, failed with error: {e}")

# Average test predictions over folds for each model
test_preds_avg = {}
for model_name in models:
    test_preds_avg[model_name] = np.mean(test_preds_cv[model_name], axis=2)  # Average over folds

# Retrain each model on the full training data and predict on the test set
print("\nRetraining models on the full training data and generating final predictions.")
final_test_preds = {}
for model_name, model in models.items():
    print(f"Retraining model: {model_name}")
    clf = model
    clf.fit(features, targets)
    y_pred_test = clf.predict(X_test)
    final_test_preds[model_name] = y_pred_test

# Combine the test predictions using the best weights
final_test_pred = np.zeros((X_test.shape[0], targets.shape[1]))
for i, model_name in enumerate(models):
    final_test_pred += best_weights[i] * final_test_preds[model_name]

# Create a DataFrame with IDs and predictions
prediction_df = pd.DataFrame({
    'ID': test_final['ID'],
    'longitude_predicted': final_test_pred[:, 1],
    'latitude_predicted': final_test_pred[:, 0]
})

# Save the predictions to a CSV file
prediction_df.to_csv('predictions_ensemble_final.csv', index=False)
print("\nPredictions saved to 'predictions_ensemble_final.csv'.")

# Print the best method and weights
print(f"\nBest optimization method: {best_method}")
print("Best weights:")
for i, model_name in enumerate(models):
    print(f"{model_name}: {best_weights[i]:.4f}")


1.9772604016895365